In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from pathlib import Path
import xarray as xr
import tensorflow as tf
from shared.models import *
from shared.utilities import *
from shared.training import train_and_evaluate, split_data_on_participants, k_fold_cross_validate, get_compile_kwargs
from shared.normalization import *
from shared.generators import *
%env TF_FORCE_GPU_ALLOW_GROWTH=true
%env TF_GPU_ALLOCATOR=cuda_malloc_async

2023-10-17 15:33:50.758520: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-17 15:33:51.350129: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


env: TF_FORCE_GPU_ALLOW_GROWTH=true
env: TF_GPU_ALLOCATOR=cuda_malloc_async


### Set up data

In [2]:
data_path = Path("data/sat1/split_stage_data.nc")

data = xr.load_dataset(data_path)

In [2]:
data_path = Path("data/sat1/split_stage_data_unprocessed_500hz.nc")

data = xr.load_dataset(data_path)

In [3]:
train_data, val_data, test_data = split_data_on_participants(data, 60, norm_dummy)

In [4]:
model = SAT1Deep(len(data.channels), len(data.samples), len(data.labels))
model.compile(**get_compile_kwargs())

In [3]:
model = SAT1TopologicalConv(5, 8, len(data.samples), len(data.labels))
model.compile(**get_compile_kwargs())

In [4]:
model = SAT1Base(len(data.channels), len(data.samples), len(data.labels))
model.compile(**get_compile_kwargs())

2023-10-17 15:33:56.165856: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-17 15:33:56.197752: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-17 15:33:56.197836: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-17 15:33:56.201380: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-17 15:33:56.201455: I tensorflow/compile

In [6]:
train_and_evaluate(
    model, train_data, val_data, test_data, epochs=20, logs_path=Path("logs/")
)

Epoch 1/20


2023-10-17 15:34:26.497756: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-10-17 15:34:27.395787: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-10-17 15:34:27.713505: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5556200fc970 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-17 15:34:27.713538: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-10-17 15:34:27.720016: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-17 15:34:27.828827: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the p

757/757 [==============================] - 16s 16ms/step - loss: 3.9849 - accuracy: 0.7077 - val_loss: 0.5730 - val_accuracy: 0.7904
Epoch 2/20
757/757 [==============================] - 12s 16ms/step - loss: 2.9211 - accuracy: 0.7929 - val_loss: 0.5205 - val_accuracy: 0.8052
Epoch 3/20
757/757 [==============================] - 12s 16ms/step - loss: 2.5946 - accuracy: 0.8076 - val_loss: 0.5396 - val_accuracy: 0.8027
Epoch 4/20
757/757 [==============================] - 12s 16ms/step - loss: 2.3006 - accuracy: 0.8259 - val_loss: 0.5285 - val_accuracy: 0.8130
Epoch 5/20
757/757 [==============================] - 12s 16ms/step - loss: 2.1000 - accuracy: 0.8464 - val_loss: 0.5234 - val_accuracy: 0.8243
Epoch 6/20
757/757 [==============================] - 12s 16ms/step - loss: 1.9111 - accuracy: 0.8596 - val_loss: 0.5214 - val_accuracy: 0.8200
Epoch 7/20
757/757 [==============================] - 12s 16ms/step - loss: 1.7053 - accuracy: 0.8719 - val_loss: 0.5899 - val_accuracy: 0.8102
Epo

(<keras.src.callbacks.History at 0x7f65cac0a9d0>,
 {'confirmation': {'precision': 0.8645598194130926,
   'recall': 0.8724373576309795,
   'f1-score': 0.8684807256235828,
   'support': 439},
  'decision': {'precision': 0.8731428571428571,
   'recall': 0.8691695108077361,
   'f1-score': 0.87115165336374,
   'support': 879},
  'encoding': {'precision': 0.8428731762065096,
   'recall': 0.8592677345537757,
   'f1-score': 0.8509915014164304,
   'support': 874},
  'pre-attentive': {'precision': 0.7732079905992949,
   'recall': 0.7750294464075382,
   'f1-score': 0.7741176470588235,
   'support': 849},
  'response': {'precision': 0.8813953488372093,
   'recall': 0.8623435722411832,
   'f1-score': 0.8717653824036803,
   'support': 879},
  'accuracy': 0.8454081632653061,
  'macro avg': {'precision': 0.8470358384397926,
   'recall': 0.8476495243282425,
   'f1-score': 0.8473013819732514,
   'support': 3920},
  'weighted avg': {'precision': 0.8456391795413563,
   'recall': 0.8454081632653061,
   'f1

In [6]:
train_and_evaluate(
    model,
    train_data,
    val_data,
    test_data,
    epochs=20,
    logs_path=Path("logs/"),
    gen_kwargs={"shape_topological": True},
)

Epoch 1/20


2023-10-16 17:56:22.072552: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-10-16 17:56:23.211803: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-10-16 17:56:23.214383: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x560c18811bc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-16 17:56:23.214413: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-10-16 17:56:23.218391: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-16 17:56:23.319523: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the p

766/766 [==============================] - 15s 15ms/step - loss: 4.6273 - accuracy: 0.6462 - val_loss: 0.7667 - val_accuracy: 0.7159
Epoch 2/20
766/766 [==============================] - 11s 14ms/step - loss: 3.8111 - accuracy: 0.7135 - val_loss: 0.7017 - val_accuracy: 0.7410
Epoch 3/20
766/766 [==============================] - 11s 14ms/step - loss: 3.4588 - accuracy: 0.7396 - val_loss: 0.7502 - val_accuracy: 0.7211
Epoch 4/20
766/766 [==============================] - 11s 14ms/step - loss: 3.2057 - accuracy: 0.7578 - val_loss: 0.7378 - val_accuracy: 0.7316
Epoch 5/20
766/766 [==============================] - 11s 14ms/step - loss: 2.9099 - accuracy: 0.7833 - val_loss: 0.7534 - val_accuracy: 0.7341
Epoch 6/20
766/766 [==============================] - 11s 14ms/step - loss: 2.7166 - accuracy: 0.7973 - val_loss: 0.7580 - val_accuracy: 0.7413
Epoch 7/20
766/766 [==============================] - 11s 14ms/step - loss: 2.4396 - accuracy: 0.8132 - val_loss: 0.7722 - val_accuracy: 0.7453
Epo

(<keras.src.callbacks.History at 0x7fcf7b960d50>,
 {'confirmation': {'precision': 0.5914893617021276,
   'recall': 0.6318181818181818,
   'f1-score': 0.610989010989011,
   'support': 440},
  'decision': {'precision': 0.6908315565031983,
   'recall': 0.7363636363636363,
   'f1-score': 0.7128712871287127,
   'support': 880},
  'encoding': {'precision': 0.7935323383084577,
   'recall': 0.728310502283105,
   'f1-score': 0.7595238095238096,
   'support': 876},
  'pre-attentive': {'precision': 0.7877777777777778,
   'recall': 0.8075170842824602,
   'f1-score': 0.797525309336333,
   'support': 878},
  'response': {'precision': 0.7857142857142857,
   'recall': 0.7517084282460137,
   'f1-score': 0.7683352735739232,
   'support': 878},
  'accuracy': 0.7421558704453441,
  'macro avg': {'precision': 0.7298690640011694,
   'recall': 0.7311435665986794,
   'f1-score': 0.7298489381103579,
   'support': 3952},
  'weighted avg': {'precision': 0.745153706724694,
   'recall': 0.7421558704453441,
   'f1-s

In [7]:
model.save("models/cnn_base_test")

INFO:tensorflow:Assets written to: models/cnn_base_test/assets


INFO:tensorflow:Assets written to: models/cnn_base_test/assets


In [12]:
# Run before re-training to clear up VRAM
import gc

gc.collect()
tf.keras.backend.clear_session()
del model

In [17]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 770, 30, 1)]      0         
                                                                 
 masking (Masking)           (None, 770, 30, 1)        0         
                                                                 
 conv2d (Conv2D)             (None, 746, 30, 64)       1664      
                                                                 
 max_pooling2d (MaxPooling2  (None, 373, 30, 64)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 357, 30, 128)      139392    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 178, 30, 128)      0         
 g2D)                                                        